In [1]:
from dh2loop import dh2l_db
import datetime
#from datetime import datetime


C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\00103098\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
%run -i "DH2_LConfig.py"

Default parameters loaded from DH2_LConfig.py:
from datetime import datetime
#Extents to query 
#ranee
minlong=115.5
maxlong=118
minlat=-30.5
maxlat=-27.5

#fabilea
#minlong=121.2
#maxlong=122.89
#minlat=-21.04
#maxlat=-21.03


#src_pro,Dst_proj
src_csr = 4326 
dst_csr = 28350


#ExportFiles
nowtime=datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")   #.isoformat(timespec='minutes')
export_path='../data/export_db/'
DB_Collar_Rl_Log = export_path + 'DB_Collar_Rl_Log_' + nowtime + '.log'
DB_Collar_Maxdepth_Log = export_path + 'DB_Collar_maxdepth_Log_' + nowtime +'.log'
DB_Collar_Export=export_path+'DB_Collar_Export_' + nowtime + '.csv'   #.replace("-","/").replace("T"," At ")
DB_Survey_Export=export_path+'DB_Survey_Export_'+ nowtime +'.csv'
DB_Survey_Export_Calc=export_path+'DB_Survey_Export_Calc_'+ nowtime +'.csv'
CET_Litho=export_path+'CET_Litho_'+ nowtime +'.csv'
DB_Lithology_Export=export_path+'DB_Lithology_Export_'+ nowtime +'.csv'
DB_Lithology_Export_Backup=export_path+'DB_Lithology_E

## Collar Extraction

In [3]:
Start_Time = datetime.now()
dh2l_db.collar_attr_col_dic()
dh2l_db.collar_collar_attri_Final(DB_Collar_Export,src_csr,dst_csr,minlong,maxlong,minlat,maxlat)
End_Time = datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

Time taken in Hour_Min_Sec_MilliSec is: 0:01:18.985951


## Survey Extraction

In [4]:
Start_Time = datetime.now()
dh2l_db.Attr_col_dic()
dh2l_db.Survey_Final(DB_Survey_Export,minlong,maxlong,minlat,maxlat)
dh2l_db.convert_survey(DB_Collar_Export,DB_Survey_Export,DB_Survey_Export_Calc)
End_Time = datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Time taken in Hour_Min_Sec_MilliSec is: 0:04:22.161012


## Lithology Extraction

In [5]:
Start_Time = datetime.now()
dh2l_db.Attr_Val_Dic()
dh2l_db.Litho_Dico()
dh2l_db.Clean_Up()
dh2l_db.Attr_val_With_fuzzy()
dh2l_db.Final_Lithology(DB_Lithology_Export,minlong,maxlong,minlat,maxlat)
#dh2l_db.Upscale_lithology(DB_Lithology_Export,DB_Lithology_Upscaled_Export)
#dh2l_db.Remove_duplicates_Litho(DB_Lithology_Upscaled_Export,Upscaled_Litho_NoDuplicates_Export)
End_Time = datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

Time taken in Hour_Min_Sec_MilliSec is: 0:20:50.192829


In [6]:
Start_Time = datetime.now()
#dh2l_db.convert_lithology()
#dh2l_db.Diff_XYZ()
End_Time = datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)


Time taken in Hour_Min_Sec_MilliSec is: 0:00:00


In [3]:
Start_Time = datetime.now()   # for logging 
dh2l_db.Attr_COl()
dh2l_db.Attr_Val_Dic()
dh2l_db.Litho_Dico()
dh2l_db.Clean_Up()
dh2l_db.First_Filter()
dh2l_db.Final_Lithology_old()
End_Time = datetime.now()
print("Time taken in Hour_Min_Sec_MilliSec is:", End_Time-Start_Time)

------------------start First_Filter------------
--------start of Final -----------
Time taken in Hour_Min_Sec_MilliSec is: 3:01:38.446804


In [21]:
import psycopg2
Attr_col_collar_dic_list =[]

query =""" SELECT  thesaurus_collar_elevation.attributecolumn,thesaurus_collar_elevation.cet_attributecolumn  FROM thesaurus_collar_elevation
           union all 
           SELECT  thesaurus_collar_maxdepth.attributecolumn,thesaurus_collar_maxdepth.cet_attributecolumn  FROM thesaurus_collar_maxdepth         """
#query = """ select * from thesaurus_collar_elevation """
conn = None
   
try:
    conn = psycopg2.connect(host="130.95.198.59", port = 5432, database="gswa_dh", user="postgres", password="loopie123pgpw")
    cur = conn.cursor()
    cur.execute(query)

    for rec in cur:
        Attr_col_collar_dic_list.append(rec)
    outputquery = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(query)
   
    with open('Dic_attr_col_collar.csv', 'w',encoding="utf-8") as f:
        cur.copy_expert(outputquery, f)
        #print(Attr_col_collar_dic_list)    
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        
        
        
        
        
        